# Tarea 1 - Fisica Computacional
#### Estudiante: Ignacio Vargas Cordero

### Ejercicio 1  
Realice un notebook en el se programen y expliquen los siguientes algoritmos de integracion de ecuaciones diferenciales:

1. Integracion con metodo de Euler
2. Integracion sobre serie de Taylor a 2 orden
3. Integracion con metodo de Runge-Kutta de 2 orden
4. Integracion con metodo de Runge-Kutta de 4 orden

Verifique el funcionamiento de sus integradores encontrando la solucion de la siguiente ecuacion diferencial para $y(t)$:

$\ddot{x} + 0.5\dot{x} + \sin{x} = e^{-t} \sin(3t)$

con condiciones iniciales $\dot{x}(0) = 1$ y $x(0) = 0$

In [5]:
from pylab import *
%matplotlib inline

In [4]:
def euler(funcion,p_ini,t_final,h=0.01):
    tiempos = arange(p_ini[0],t_final+h,h)
    y = [0]*len(tiempos) #zeros(len(tiempos))
    y[0] = p_ini[1]
    for i in range(len(tiempos)-1):
        y[i+1] = y[i] + funcion(y[i],tiempos[i])*h
    solucion = [tiempos,y]
    return solucion

def integra2(ec,d_ec,p_ini,x,h=0.1):
    tiempos = arange(x[0],x[1]+h,h)
    sol = zeros((len(tiempos),len(p_ini)))
    sol[0,:] = p_ini
    for i in xrange(len(tiempos)-1):
        f = ec(sol[i,:],tiempos[i])
        df,jac=d_ec(sol[i,:],tiempos[i])
        sol[i+1,:]=sol[i,:]+h*f+(0.5*h*h*(df+dot(jac,f)))
    return tiempos,sol

def rk2(ec,p_ini,x,h=0.1):
    tiempos = arange(x[0],x[1]+h,h)
    sol = zeros((len(tiempos),len(p_ini)))
    sol[0,:] = p_ini
    for i in xrange(len(tiempos)-1):
        k = h*ec(sol[i,:],tiempos[i])
        f = ec(sol[i,:]+0.5*k,tiempos[i]+0.5*h)
        sol[i+1,:]=sol[i,:]+h*f
    return tiempos,sol

def rk4(ec,p_ini,x,h=0.1):
    tiempos = arange(x[0],x[1]+h,h)
    sol = zeros((len(tiempos),len(p_ini)))
    sol[0,:] = p_ini
    for i in xrange(len(tiempos)-1):
        k1 = ec(sol[i,:],tiempos[i])
        k2 = ec(sol[i,:]+0.5*h*k1,tiempos[i]+0.5*h)
        k3 = ec(sol[i,:]+0.5*h*k2,tiempos[i]+0.5*h)
        k4 = ec(sol[i,:]+h*k3,tiempos[i]+h)
        sol[i+1,:]=sol[i,:]+(h/6.)*(k1+2*k2+2*k3+k4)
    return tiempos,sol

In [6]:
def dif1(x,t):
    return (d^2 x(t))/(dt^2)+0.5*(dx(t))/(dt)+sin(x(t)) = e^(-t) sin(3 t)

SyntaxError: invalid syntax (<ipython-input-6-4ab9893d5563>, line 2)

### Ejercicio 2  
Utilizando los integradores anteriores encuentre el mejor paso de integracion para la siguiente ecuacion:

$\dddot{y} + \ddot{y}^2 - 3\dot{y}^3 + \cos^2{y} = e^{-t} \sin(3t)$

con condiciones iniciales $\ddot{y}(1) = 1, \dot{y}(1) = 2, y(1) = 1$. En particular que sucede en el intervalo $t \in [1,2]$ y $t \in [1,2.1]$.

### Ejercicio 3  
Modifique las funciones integradoras en el problema 1 de tal manera que se puedan pasar argumentos opcionales al sistema de ecuaciones diferenciales de tal manera que se puedan modificar parametros al momento de hacer la integracion. Una vez hecho esto, obtenga la solucion de la siguiente ecucacion diferencial

$\ddot{x} + \frac{1}{10}\dot{x} + 4\sin{x} = \frac{1}{2}\sin(\omega t)$

Con condiciones iniciales $\dot{x}(0) = 1$ y $x(0) = 0$ y para $\omega = \{1,1.1,1.2,1.3,...,2.9,3\}$. Realice la grafica de algunas soluciones que representen el comportamiento de la solucion en funcion de $\omega$ y explique su resultado.


### Ejercicio 4  
Considere el siguiente sistema de ecuaciones diferenciales

$$\frac{dx}{dt} = 10(y-x)\\
\frac{dy}{dt} = x(28-z)-y\\
\frac{dx}{dt} = xy - \frac{8}{3}z$$

Realice una integracion numerica del sistema de ecuaciones con alguno de los integradores del problema (el que mas le guste) y realice los siguientes ejercicios:

1. Elja una condicion inicial aleatoria en el $x,y,z \in [-5,5]$ y realice la grafica de la solucion en el plano $xy, xz, yz$ y comente sobre sus observaciones.

2. Ahora tome 3 soluciones parecidas. Para ello proponga 3 condiciones iniciales tales que en la coordenada en $y$ las condiciones iniciales difieran entr esi en $10^{-2}$. Explique lo que observa.  
Nota: De preferencia realice las integraciones con tiempos relativamente largos en los que pueda apreciar diferencias en el comportamiento de las soluciones.